In [ ]:
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage,RSI
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.pipeline.factors import CustomFactor,RSI
from quantopian.pipeline.data import morningstar
from zipline import TradingAlgorithm  
from quantopian.pipeline.filters import Q1500US, Q500US
from quantopian.pipeline.factors import AverageDollarVolume

from quantopian.pipeline.classifiers.morningstar import Sector, SuperSector


import numpy as np
import talib 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.cm as cm


class PrevClose(CustomFactor):
    inputs = [USEquityPricing.close]
    window_length = 2
    def compute(self, today, assets, out, close):
        out[:] = close[-1]
        
class PrevVolume(CustomFactor):
    inputs = [USEquityPricing.volume]
    window_length = 2
    def compute(self, today, assets, out, close):
        out[:] = close[-1]
        
def make_pipeline():
    base_universe = Q500US()
    yesterday_close = PrevClose()
    yesterday_volume = PrevVolume()
    dollar_volume = AverageDollarVolume(window_length=30)
    #ToDo この範囲を色々変えてみる．
    high_dollar_volume = dollar_volume.percentile_between(98, 100)
    sector = Sector()
    rsi = RSI(inputs=[USEquityPricing.close])

    
    pipe = Pipeline(
        
        columns = {
            'yesterday_close': yesterday_close,
            'yesterday_volume': yesterday_volume,
            'yesterday_turnover': yesterday_close * yesterday_volume,
            'dollar_volume': dollar_volume,
            'high_dollar_volume': high_dollar_volume, 
            'sector': sector,
            'rsi': rsi, 
        },
        screen = base_universe & high_dollar_volume #& rsi_under_60
    )
    return pipe

def get_prices(pipeline_results, date):
    sids = pipeline_results.loc[date].index
    pan_today_1m = get_pricing(sids, start_date=date, end_date=date, frequency='minute') 
    pan_today_1m['turnover'] = pan_today_1m.price * pan_today_1m.volume
    return pan_today_1m 

def calc_gap(df_pipeline_results_prevday, pan_today_1m, observe_timing, ):
    """
    pipeline_results: items: Datetime / major_axis: SID / minor_axis: pipelineのcolumns 
    pan_today_1mは，0930-1600のデータが入っている
    """
    
    s_latest_price = pan_today_1m['price', observe_timing, :] #pan_today_1m.price.ix[observe_timing]
    s_latest_turnover = pan_today_1m['turnover', observe_timing, :] #pan_today_1m.turnover.ix[observe_timing]
    
    s_turnover = df_pipeline_results_prevday.yesterday_turnover
    s_prev_close = df_pipeline_results_prevday.yesterday_close
    
    df_eligibles = pd.DataFrame({
        'gap': s_latest_price / s_prev_close - 1.0,
        'turnover_ratio': s_latest_turnover/s_turnover,
        'rsi': df_pipeline_results_prevday.rsi,
        'sector': df_pipeline_results_prevday.sector, 
        'latest_turnover':s_latest_turnover, 
    })
    return df_eligibles
    
def find_gapups(df_eligibles, 
                gapup_min_turnover_ratio,
                gapup_max_turnover_ratio,
                gapup_min_gap, 
                gapup_max_gap,):
    
    df_gapups = df_eligibles[
        (df_eligibles.turnover_ratio > gapup_min_turnover_ratio)
        & (df_eligibles.turnover_ratio < gapup_max_turnover_ratio)
        & (df_eligibles.gap > gapup_min_gap )
        & (df_eligibles.gap < gapup_max_gap )
    ].sort_values(by=['gap'], ascending=[False])
    
    return df_gapups
  


In [ ]:

pipeline_results = run_pipeline(make_pipeline(), start_date='2017-1-1', end_date='2017-12-12')
dates = pipeline_results.index.get_level_values(0).unique()
observe_timing = 1


d = dict()
d_pan_today_1m = dict()
for date in dates:
    print date.strftime("%Y-%m-%d"),
    df_pipeline_results_prevday = pipeline_results.ix[date]   
    pan_today_1m = get_prices(pipeline_results, date)
    df_eligibles = calc_gap(df_pipeline_results_prevday, pan_today_1m, observe_timing,)
    d[date] = df_eligibles
    d_pan_today_1m[date] = pan_today_1m

    

In [ ]:
gapup_min_turnover_ratio = 0.0
gapup_max_turnover_ratio = 1.0
gapup_min_gap = -1.0
gapup_max_gap = 1.0
r = dict()
spy = get_pricing(symbols('spy'), start_date=dates[0], end_date=dates[-1], frequency='daily') 
spy['gap'] =  spy.open_price / spy.close_price.shift(1) - 1

for date in dates:
    df_eligibles = d[date]
    df_gapups = find_gapups(df_eligibles,
                               gapup_min_turnover_ratio,
                               gapup_max_turnover_ratio,
                               gapup_min_gap, 
                               gapup_max_gap)
    r[date] = df_gapups



In [ ]:
l = []
l2 = []
l3 = []
import matplotlib.cm as cm

special_dates = dates#[("2014-1-1" < dates) & (dates < "2015-1-1") & (6 < dates.month) & ( dates.month <= 12)]
for date in special_dates:
    df = d_pan_today_1m[date]['price',:,r[date].index]
    if not df.empty:
        df = df.reset_index(drop=True)
        df = df.pct_change().cumsum()
        l.append(pd.DataFrame({'gap':r[date].gap, 
                               'turnover_ratio':r[date].turnover_ratio, 
                               'sector':r[date].sector, 
                               'latest_turnover':r[date].latest_turnover.apply(np.log),
                               '05m':df.ix[5],
                               '10m':df.ix[10],
                               '15m':df.ix[15],
                               '20m':df.ix[20],
                               '25m':df.ix[25],
                               '30m':df.ix[30],
                               '35m':df.ix[35],
                               '40m':df.ix[40],
                               '45m':df.ix[45],
                               '50m':df.ix[50],
                               'spy':spy['gap'].ix[date],
                               'date':date
                              }))
        
x = pd.concat(l)
#plt.scatter(x['gap'],x['5m'], s=2, color='blue')
thresh = 0.01
#margin = 0.01
x = x[(x.spy < -thresh) | (x.spy > thresh) ]
x = x[(x.sector != 102) & (x.sector != 105) & (x.sector != 301)]
#x = x[(x.gap < x.spy-margin) | (x.gap > x.spy+margin) ]
fig = plt.figure()
#plt.xlim([-0.02, 0.0])
#plt.ylim([-0.05, 0.05])

Y = '20m'
sx = x['gap']-x['spy']*1.02
sy = x[Y]

sz = x['turnover_ratio'].apply(np.log)
#sz = x['latest_turnover']
#sz = x['sector']

plt.xlabel('gap-sp/gap')
plt.ylabel(Y)

im = plt.scatter(sx,
                 sy,
                 #s=5, 
                 c=sz , ## 配色を決定する三番目のデータ
                 linewidths=0, alpha=1, 
                 cmap=cm.coolwarm, # ここでカラーマップを指定
                 #vmin=0.94, 
                 #vmax=0.003,
                )
fig.colorbar(im)


In [ ]:
x[x.columns[:-5]][x.gap>0].describe().ix['mean'].plot(kind='bar')
#x.date.unique()


In [ ]:
len(spy.head(-1)), len(spy)


In [ ]:

spy

In [ ]:
def find_gapdown2(results, pan, date, turnover_threshold=0.0, gapdown_threshold = 0.0):
    pan['interaday'] = pan.open_price / pan.open_price.ix[0] - 1
    df = pd.DataFrame({'gap': (pan.price.ix[0] / results.ix[date].yesterday_close - 1),
                       'turnover': (pan.turnover.ix[0] / results.ix[date].yesterday_turnover), 
                      'std': pan.interaday.ix[:10].std(), 
                       'ten_min': pan.interaday.ix[10]
                      })
    df['filled'] = df.gap + df.ten_min
    
    df_gapdowns = df[(df.turnover > turnover_threshold) & (df.gap < gapdown_threshold) & (df.filled>-0.005)]
    df_gapdowns = df_gapdowns.sort_values(by=['gap','turnover', 'std'], ascending=[True,False, False])
    return df_gapdowns
    


#pan = my_get_pricing(results, dates[0])
dates[-2]

In [ ]:
find_gapdown2(results, pan, dates[-19])
#pan.fillgap.ix[:10].std().sort_values(ascending=False)

In [ ]:
bt = get_backtest("593c9824a3db0e6961c6ce6d")
bt.create_full_tear_sheet()